In [1]:
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
from qiskit_ibm_runtime import Estimator
from qiskit.quantum_info import SparsePauliOp

In [2]:
qc = QuantumCircuit(3)
qc.h(2)
qc.cx(2, 1)
qc.x(0)
qc.cx(0, 2)

# Define the measurement circuit
# It transforms Z basis to X basis
qc_mes = QuantumCircuit(3)
qc_mes.h(0)
qc_mes.h(1)
qc_mes.h(2)

# Combine base with measurement circuit
qc_all = qc.copy()
qc_all.compose(qc_mes, inplace=True)

num_shots = 1024
backend = AerSimulator()

# Use estimator to run the circuit
estimator = Estimator(backend, options={"default_shots": num_shots})

op = SparsePauliOp.from_list(
    [
        ("XXX", (-105.8)),
    ]
)
job = estimator.run([(qc, op)])
measured_energy = job.result()[0].data.evs
print('estimator expectation val:', measured_energy)

# Use simulator to run the circuit
qc_all.measure_all()
job2 = backend.run(qc_all, shots=num_shots)

job2_res = job2.result().get_counts()
exp_cal = 0

for key, val in job2_res.items():
    if key.count('1') % 2 == 0:
        exp_cal += val
    else:
        exp_cal -= val
exp_cal = exp_cal * (-105.8) / num_shots
print('calculated expectation val:', exp_cal)

estimator expectation val: 5.7859375
calculated expectation val: -4.1328125


In [14]:
from qiskit import QuantumCircuit
from qiskit_aer import Aer
from qiskit.primitives import BackendEstimatorV2 as BackendEstimator
from qiskit.quantum_info import SparsePauliOp

In [ ]:
# Define circuit
qc = QuantumCircuit(2)
qc.h(0)
qc.crx(0.5, 0, 1)

observable = SparsePauliOp.from_list([("ZI", 1.0)])

# Use AerSimulator backend
backend = Aer.get_backend("aer_simulator")

# Create estimator with shot noise
#estimator = BackendEstimator(backend=backend, options={"shots": 1000, "seed_simulator": 42})
estimator = BackendEstimator(backend=backend)

# Run and get result
#job = estimator.run([qc], [observable])
job = estimator.run([(qc, observable)])
result = job.result()
print("⟨Z₀⟩ ≈", result.values[0])

TypeError: AerBackend.set_options() takes 1 positional argument but 2 were given

In [17]:
result

PrimitiveResult([PubResult(data=DataBin(evs=np.ndarray(<shape=(), dtype=float64>), stds=np.float64(0.0056973496598593685)), metadata={'target_precision': 0.015625, 'shots': 4096, 'circuit_metadata': {}})], metadata={'version': 2})

In [ ]:
 DeprecationWarning: The class ``qiskit.primitives.backend_estimator.BackendEstimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `BackendEstimator` class is `BackendEstimatorV2`.
  estimator = BackendEstimator(backend=backend, options={"shots": 1000, "seed_simulator": 42})